<a href="https://colab.research.google.com/github/tutsilianna/Greetings_and_Wishes_Generator/blob/main/Greetings_and_Wishes_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Клонирование репозитория с проектом**

In [1]:
! git clone https://github.com/tutsilianna/Greetings_and_Wishes_Generator.git

Cloning into 'Greetings_and_Wishes_Generator'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 11), reused 11 (delta 2), pack-reused 0
Receiving objects: 100% (30/30), 8.79 MiB | 8.26 MiB/s, done.
Resolving deltas: 100% (11/11), done.


# **Подключение библиотек**

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd

from huggingface_hub import notebook_login
notebook_login()

# **Обработка данных**

In [3]:
data = pd.read_csv('/content/Greetings_and_Wishes_Generator/data/raw/greetings.csv', encoding='utf-8')
data.drop_duplicates(inplace=True)
data.fillna('', inplace=True)
len(data)

27201

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27201 entries, 0 to 95219
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         27201 non-null  object
 1   likes        27201 non-null  object
 2   holiday      27201 non-null  object
 3   to           27201 non-null  object
 4   description  27201 non-null  object
dtypes: object(5)
memory usage: 1.2+ MB


In [5]:
data = data.loc[(data['holiday'] != "Сценарии") & (data['holiday'] != "Конкурсы") & (data['holiday'] != "Розыгрыши")]
data['holiday'].value_counts()

holiday
На день рождения                  9150
На праздники                      7356
СМС                               3646
Тосты                             2125
На свадьбу                         938
Ежедневные стихи                   899
Важные события                     840
По профессиям                      836
На работу и учебу                  612
Поздравления детей и для детей     200
Признания и предложения            160
Извинения                           80
Name: count, dtype: int64

In [6]:
len(data)

26842

In [7]:
data['promt']   = 'напиши поздравление ' + data['holiday'].str.lower() + ' ' + data['to'].str.lower()
data['promt'] = data.apply(lambda row: row['promt'] + ' ' + row['description'].lower() if row['to'].lower() not in row['description'].lower() else row['promt'], axis=1)
data['text'] = data['text'].str.replace(r"\xao", ' ')

data.head()

,text,likes,holiday,to,description,promt
0,"Желаем в жизни лишь удачи\nИ достижения мечты,...",6084.0,На день рождения,Подруге,Красивые,напиши поздравление на день рождения подруге к...
1,"Ты сегодня на год повзрослела,\nК зеркалу с тр...",4080.0,На день рождения,Подруге,Красивые,напиши поздравление на день рождения подруге к...
2,За доброе сердце твоё и за нежность\nТы Богом ...,3736.0,На день рождения,Подруге,Красивые,напиши поздравление на день рождения подруге к...
3,Что пожелать тебе сегодня?\nТак много хочется ...,2766.0,На день рождения,Подруге,Красивые,напиши поздравление на день рождения подруге к...
4,"С каждым Днем Рождения\nОпыт прибавляется,\nПл...",2472.0,На день рождения,Подруге,Красивые,напиши поздравление на день рождения подруге к...


In [8]:
print('Пример поздравления: \n')
print(data['text'][1456])

Пример поздравления: 

Желаю верных я тебе друзей, надежных,
Пусть не способны они будут на обман,
И достижений в жизни даже невозможных,
А беды испарятся все, словно туман.

Тебя, Подруга, поздравляю с юбилеем,
Пятьдесят пятый день рожденье у тебя,
Пускай сопутствует успех и сердце греет,
И на лице улыбка светится твоя.
1


In [9]:
def remove_last_line_if_digits(text):
    lines = text.split('\n')
    last_line = lines[-1]
    if last_line.isdigit():
        return '\n'.join(lines[:-1])
    else:
        return text

data['text'] = data['text'].apply(remove_last_line_if_digits)

print('Пример поздравления: \n')
print(data['text'][1456])

Пример поздравления: 

Желаю верных я тебе друзей, надежных,
Пусть не способны они будут на обман,
И достижений в жизни даже невозможных,
А беды испарятся все, словно туман.

Тебя, Подруга, поздравляю с юбилеем,
Пятьдесят пятый день рожденье у тебя,
Пускай сопутствует успех и сердце греет,
И на лице улыбка светится твоя.


In [10]:
data_ = data[['promt', 'text']]
data_.head(10)

,promt,text
0,напиши поздравление на день рождения подруге к...,"Желаем в жизни лишь удачи\nИ достижения мечты,..."
1,напиши поздравление на день рождения подруге к...,"Ты сегодня на год повзрослела,\nК зеркалу с тр..."
2,напиши поздравление на день рождения подруге к...,За доброе сердце твоё и за нежность\nТы Богом ...
3,напиши поздравление на день рождения подруге к...,Что пожелать тебе сегодня?\nТак много хочется ...
4,напиши поздравление на день рождения подруге к...,"С каждым Днем Рождения\nОпыт прибавляется,\nПл..."
5,напиши поздравление на день рождения подруге к...,"Ах, как природа поступила справедливо,\nКогда ..."
6,напиши поздравление на день рождения подруге к...,"С днем рожденья тебя поздравляю,\nТы прочти по..."
7,напиши поздравление на день рождения подруге к...,Пусть будут у моей подружки\nЗолотые побрякушк...
8,напиши поздравление на день рождения подруге к...,Подруга милая! Удачи!\nПусть все исполнятся ме...
9,напиши поздравление на день рождения подруге к...,С днем рожденья поздравляем!\nИ от всей души ж...


In [11]:
def get_txt(data, name):
    with open(f'/content/Greetings_and_Wishes_Generator/data/processed/{name}.txt', 'w') as file:
        for i in range(len(data)):
            str_ = '<s>' + data['promt'].iloc[i] + ':\n' + data['text'].iloc[i] + '</s>'
            if i == len(data) - 1:
                pass
            else:
                str_ += '\n'
            file.write(str_)

train, valid = train_test_split(data_, test_size=0.2, random_state=42, stratify=data_['promt'])
get_txt(train, 'train')
get_txt(valid, 'valid')

In [12]:
data_dict = data_.groupby('promt')['text'].apply(list).to_dict()

# Обучение

In [13]:
train = {
  'model_type': 'gpt2',
  'train_size': 0.9,
  'per_gpu_batch_size': 1,
  'gradient_steps': 1,
  'num_train_epochs': 5,
  'block_size': 150
}

path = {
  'model_load':  'sberbank-ai/rugpt3large_based_on_gpt2',
  'model_write':  'data/model',
  'pretrain':    '/content/ru-gpts/pretrain_transformers.py',
  'train':       '/content/Greetings_and_Wishes_Generator/data/processed/train.txt',
  'val':         '/content/Greetings_and_Wishes_Generator/data/processed/valid.txt',
  'dataset':     '/content/Greetings_and_Wishes_Generator/data/raw/greetings.csv',
  'test_promt':  '/content/Greetings_and_Wishes_Generator/data/greetings_promt.txt',
  'test_text':   '/content/Greetings_and_Wishes_Generator/data/greetings_text.txt'
}

In [14]:
#### generator.py

from transformers import GPT2LMHeadModel, GPT2Tokenizer

class GreetingsGenerator:

    def __init__(self, path: str, use_cuda=False) -> None:
        """
        Инициализация модели по весам, расположенным в
        директории path, и гиперпараметров для нее
        """
        self.max_length = 700
        self.repetition_penalty = 1.1
        self.do_sample = True
        self.top_k = 5
        self.top_p = 1
        self.temperature = 1
        self.stop_token = '</s>'
        self.tok = GPT2Tokenizer.from_pretrained(path)
        self.model = GPT2LMHeadModel.from_pretrained(path).cuda()
        self.use_cuda = use_cuda
        if self.use_cuda:
            self.model = self.model.cuda()

    def update_params(self, **kwargs) -> None:
        """
        Обновляет параметры нашей модели и выдает ошибку,
        если соответствующего параметра нет
        """
        for key, value in kwargs.items():
            params = set(self.__dict__.keys()) - \
                     set(["model", "use_gpu", "tok"])
            if key not in params:
                raise Exception(f"Invalid key: {key}")
            self.__dict__[key] = value


    def generate(self, promt: str) -> str:
        """
        Возвращает строку, которая содержит текст поздравления,
        сгенерированный по заданному промту promt
        """
        # токенизириуем входной текст
        text = f"<s>{promt}\n: "
        input = self.tok.encode(text, return_tensors="pt")

        # собираем словарь с параметрами для gpt2 модели
        # это все поля класса кроме model и token
        params = {x: self.__dict__[x] \
            for x in self.__dict__ if x not in ["model", "tok",'stop_token', 'use_cuda']}

        # даем входной ембеддинг и параметры для нашей модельки
        if self.use_cuda:
            input = input.cuda()
        out = self.model.generate(input, **params)
        output_text = self.tok.decode(out[0])

        try:
            # оказывается, что модель выдает поздравление не только по
            # заданному промту, но и по другим рандомным промтам.
            # поэтому урезаем все что идет после стоп-токена </s>
            res = str(output_text[:output_text.index("</s>") + 1])[:-1]
        except:
            # если стоп токена </s> нет, значит текст содержит более
            # max_tokens токенов в таком случае оставляем все как есть
            res = output_text
        # убираем первые две строчки содержащие промт
        res = res.split("\n", 2)[2]
        return res

In [ ]:
#### train.py

train_path          = path['train']
val_path            = path['val']
pretrain_path       = path['pretrain']
model_load_path     = path['model_load']
model_write_path    = path['model_write']
model_type          = train['model_type']
per_gpu_batch_size  = train['per_gpu_batch_size']
gradient_steps      = train['gradient_steps']
num_train_epochs    = train['num_train_epochs']
block_size          = train['block_size']

!git clone https://github.com/sberbank-ai/ru-gpts


In [ ]:
%%writefile setup.sh

! export CUDA_HOME=/usr/local/cuda-10.1
! git clone https://github.com/NVIDIA/apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

!sh setup.sh

In [ ]:
# обучение
!python {pretrain_path} \
    --output_dir=model_write_path \
    --model_type={model_type} \
    --model_name_or_path={model_load_path}  \
    --do_train \
    --train_data_file={train_path} \
    --do_eval \
    --eval_data_file={val_path} \
    --per_gpu_train_batch_size {per_gpu_batch_size} \
    --gradient_accumulation_steps {gradient_steps} \
    --num_train_epochs {num_train_epochs} \
    --block_size {block_size} \
    --overwrite_output_dir

In [ ]:
#### inference.py

model_path          = path['model_load']
test_title_path     = path['test_promt']
test_text    = path['test_text']

# загружаем генератор из директории model_path
generator = GreetingsGenerator(model_path, use_cuda=True)

# меняем один гиперпараметр
generator.update_params(temperature=1.1)

# сохраняем промт из файла в test_title в promt
with open(test_title_path, "r", encoding="utf-8") as f:
    promt = f.read().splitlines()[0]

# генерируем поздравление по промту promt и сохраняем в test_text
lyrics = generator.generate(promt)
with open(test_text, "w", encoding="utf-8") as f:
    f.write(lyrics)